In [1]:
from DiscretizationSchemes import discretizationScheme, StockpriceGBM # consider different discretization schemes?
from intrinsicOptionValues import call, put # consider more payoffs?


# Page 18 (Lokeshwar): Bermudan put on a single asset

In [2]:
# set parameters
S0, K, r, sigma, T, M = 40, 40, 0.06, 0.2, 1, 10
p = 32
Np = 3*p + 1
mu = r # changing this should not make a difference??

In [3]:
# define phi for 1-dimensional vector
def phi(x: float):
  return max(x, 0)

In [4]:
# define output network
def G(beta_tm, S_tm):
  network_output = beta_tm[3*p]
  for i in range(p):
      network_output += beta_tm[2*p + i]*phi(beta_tm[i]*S_tm + beta_tm[p+i]) 

In [5]:
from helpers import forward, bs_put, bs_call
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [6]:
# compute continuation value Qtm (for ease of notation we write tm where the book writes t_{m-1})
def Qt_mminus1(beta_tm, S_tm, Deltat_mplus1):
  """
  Deltat_mplus1 = t_{m+1} - t_m
  """
  sumCondExp = beta_tm[3*p]
  for i in range(p):
    w_i, b_i, omega_i = beta_tm[i], beta_tm[p+i], beta_tm[2*p+i]
    if w_i >= 0 and b_i >= 0:
      condExp = forward(w_i*S_tm + b_i, r, Deltat_mplus1)
    elif w_i >= 0 and b_i < 0:
      condExp = w_i * bs_call(S_tm, -b_i/w_i, Deltat_mplus1, r, sigma)
    elif w_i < 0 and b_i >= 0:
      condExp = -w_i * bs_put(S_tm, -b_i/w_i, Deltat_mplus1, r, sigma)
    elif w_i < 0 and b_i < 0:
      condExp = 0
    sumCondExp += omega_i*condExp
  return sumCondExp


In [7]:
monitoring_dates = np.linspace(0,T,M)
N = 10000

def h(S):
  return K-S

In [8]:
def fitBeta(S_tm, V_tm, load_weights):
  x = S_tm
  y = V_tm

  model = keras.Sequential()  
  kernel_init = tf.keras.initializers.RandomUniform(minval=0., maxval=1.)
  bias_init = tf.keras.initializers.RandomUniform(minval=0., maxval=1.)
  model.add(keras.layers.Dense(p, input_dim=1, kernel_initializer=kernel_init, use_bias=True, bias_initializer=bias_init, activation='relu'))

  output_layer = keras.layers.Dense(1, kernel_initializer=kernel_init, use_bias=True, bias_initializer=bias_init, activation='linear')
  model.add(output_layer)
  
  opt = keras.optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer=opt,loss='mean_squared_error')
  if load_weights==True:
    model.load_weights('model_weights.h5')
  model.fit(x, y, epochs=1000, batch_size=int(N/10))
  model.save_weights('model_weights.h5')
  return model

In [9]:
# generate paths
S = []
for n in range(N):
  path = discretizationScheme(monitoring_dates, S0, mu, sigma)
  S.append(path)
S = np.matrix(S) # (n,m)

# evaluate final time option value
V = np.zeros((N,M))
for n in range(N):
  S_T = S[n,M-1]
  V[n,M-1] = max(h(S_T), 0)

# keep track of Q to see what goes wrong
holdValues = np.zeros((N,M-1))

# initialize beta_tM
beta_tM = np.random.uniform(0,1,Np)
load_weights = False

for m in range(M,0,-1):
  # fit the network
  fit_tm = fitBeta(S[:,m-1],V[:,m-1],load_weights=load_weights)
  load_weights = True
  w1 = np.array(fit_tm.get_weights()[0][0])
  b1 = np.array(fit_tm.get_weights()[1])
  w2 = np.array(fit_tm.get_weights()[2]).flatten()
  b2 = np.array(fit_tm.get_weights()[3])
  beta_tm = np.concatenate((w1,b1,w2,b2))
  # stop when t=t0
  if m==1:
    break
  for n in range(N):
    Deltat_m = monitoring_dates[m-1] - monitoring_dates[m-2]
    St_mminus1 = S[n,m-2]
    holdValue = Qt_mminus1(beta_tm, St_mminus1, Deltat_m)
    holdValues[n, m-2] = holdValue
    V[n,m-2] = max(h(St_mminus1), holdValue)


/Users/pieterout/Documents/SemiStaticReplication/randomName/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Epoch 1/1000
10/10 [==============================] - 0s 597us/step - loss: 145576.5469
Epoch 2/1000
10/10 [==============================] - 0s 491us/step - loss: 136210.7031
Epoch 3/1000
10/10 [==============================] - 0s 549us/step - loss: 127448.4609
Epoch 4/1000
10/10 [==============================] - 0s 498us/step - loss: 119290.2656
Epoch 5/1000
10/10 [==============================] - 0s 540us/step - loss: 111718.5312
Epoch 6/1000
10/10 [==============================] - 0s 467us/step - loss: 104700.1172
Epoch 7/1000
10/10 [==============================] - 0s 469us/step - loss: 98204.6875
Epoch 8/1000
10/10 [==============================] - 0s 475us/step - loss: 92177.7500
Epoch 9/1000
10/10 [==============================] - 0s 467us/step - loss: 86581.5703
Epoch 10/1000
10/10 [==============================] - 0s 447us/step - loss: 81386.2812
Epoch 11/1000
10/10 [==============================] - 0s 449us/step - loss: 76551.7734
Epoch 12/1000
10/10 [==============

Epoch 1/1000
10/10 [==============================] - 0s 512us/step - loss: 0.8980
Epoch 2/1000
10/10 [==============================] - 0s 466us/step - loss: 0.8554
Epoch 3/1000
10/10 [==============================] - 0s 447us/step - loss: 0.8452
Epoch 4/1000
10/10 [==============================] - 0s 445us/step - loss: 0.8442
Epoch 5/1000
10/10 [==============================] - 0s 479us/step - loss: 0.8398
Epoch 6/1000
10/10 [==============================] - 0s 438us/step - loss: 0.8406
Epoch 7/1000
10/10 [==============================] - 0s 438us/step - loss: 0.8377
Epoch 8/1000
10/10 [==============================] - 0s 453us/step - loss: 0.8340
Epoch 9/1000
10/10 [==============================] - 0s 452us/step - loss: 0.8379
Epoch 10/1000
10/10 [==============================] - 0s 423us/step - loss: 0.8310
Epoch 11/1000
10/10 [==============================] - 0s 405us/step - loss: 0.8282
Epoch 12/1000
10/10 [==============================] - 0s 423us/step - loss: 0.8215
E

Epoch 1/1000
10/10 [==============================] - 0s 511us/step - loss: 41.2202
Epoch 2/1000
10/10 [==============================] - 0s 441us/step - loss: 37.8724
Epoch 3/1000
10/10 [==============================] - 0s 466us/step - loss: 37.6364
Epoch 4/1000
10/10 [==============================] - 0s 478us/step - loss: 37.5142
Epoch 5/1000
10/10 [==============================] - 0s 474us/step - loss: 37.3832
Epoch 6/1000
10/10 [==============================] - 0s 456us/step - loss: 37.3289
Epoch 7/1000
10/10 [==============================] - 0s 456us/step - loss: 37.2044
Epoch 8/1000
10/10 [==============================] - 0s 452us/step - loss: 37.0980
Epoch 9/1000
10/10 [==============================] - 0s 444us/step - loss: 37.0419
Epoch 10/1000
10/10 [==============================] - 0s 450us/step - loss: 37.0107
Epoch 11/1000
10/10 [==============================] - 0s 446us/step - loss: 36.8263
Epoch 12/1000
10/10 [==============================] - 0s 422us/step - los

Epoch 1/1000
10/10 [==============================] - 0s 528us/step - loss: 2454.1982
Epoch 2/1000
10/10 [==============================] - 0s 470us/step - loss: 2145.1965
Epoch 3/1000
10/10 [==============================] - 0s 517us/step - loss: 2147.8447
Epoch 4/1000
10/10 [==============================] - 0s 495us/step - loss: 2119.0776
Epoch 5/1000
10/10 [==============================] - 0s 526us/step - loss: 2110.6758
Epoch 6/1000
10/10 [==============================] - 0s 471us/step - loss: 2110.0857
Epoch 7/1000
10/10 [==============================] - 0s 515us/step - loss: 2105.2717
Epoch 8/1000
10/10 [==============================] - 0s 535us/step - loss: 2104.1443
Epoch 9/1000
10/10 [==============================] - 0s 476us/step - loss: 2101.9043
Epoch 10/1000
10/10 [==============================] - 0s 554us/step - loss: 2100.0728
Epoch 11/1000
10/10 [==============================] - 0s 463us/step - loss: 2098.1858
Epoch 12/1000
10/10 [==============================]

...

Epoch 1/1000
10/10 [==============================] - 0s 515us/step - loss: 5193300.0000
Epoch 2/1000
10/10 [==============================] - 0s 458us/step - loss: 4635727.0000
Epoch 3/1000
10/10 [==============================] - 0s 469us/step - loss: 4149213.2500
Epoch 4/1000
10/10 [==============================] - 0s 433us/step - loss: 3733835.2500
Epoch 5/1000
10/10 [==============================] - 0s 450us/step - loss: 3388367.5000
Epoch 6/1000
10/10 [==============================] - 0s 458us/step - loss: 3109990.5000
Epoch 7/1000
10/10 [==============================] - 0s 450us/step - loss: 2887806.5000
Epoch 8/1000
10/10 [==============================] - 0s 499us/step - loss: 2713959.2500
Epoch 9/1000
10/10 [==============================] - 0s 461us/step - loss: 2582014.2500
Epoch 10/1000
10/10 [==============================] - 0s 448us/step - loss: 2484148.0000
Epoch 11/1000
10/10 [==============================] - 0s 464us/step - loss: 2412161.5000
Epoch 12/1000
10/10

Epoch 1/1000
10/10 [==============================] - 0s 516us/step - loss: 72620408.0000
Epoch 2/1000
10/10 [==============================] - 0s 467us/step - loss: 68074104.0000
Epoch 3/1000
10/10 [==============================] - 0s 489us/step - loss: 63705016.0000
Epoch 4/1000
10/10 [==============================] - 0s 453us/step - loss: 59527124.0000
Epoch 5/1000
10/10 [==============================] - 0s 467us/step - loss: 55545200.0000
Epoch 6/1000
10/10 [==============================] - 0s 435us/step - loss: 51760428.0000
Epoch 7/1000
10/10 [==============================] - 0s 470us/step - loss: 48169504.0000
Epoch 8/1000
10/10 [==============================] - 0s 447us/step - loss: 44768172.0000
Epoch 9/1000
10/10 [==============================] - 0s 463us/step - loss: 41551012.0000
Epoch 10/1000
10/10 [==============================] - 0s 451us/step - loss: 38513572.0000
Epoch 11/1000
10/10 [==============================] - 0s 429us/step - loss: 35651080.0000
Epoch 12

Epoch 1/1000
10/10 [==============================] - 0s 501us/step - loss: 5418942.0000
Epoch 2/1000
10/10 [==============================] - 0s 481us/step - loss: 4156437.2500
Epoch 3/1000
10/10 [==============================] - 0s 462us/step - loss: 3095377.5000
Epoch 4/1000
10/10 [==============================] - 0s 452us/step - loss: 2237033.5000
Epoch 5/1000
10/10 [==============================] - 0s 465us/step - loss: 1568141.5000
Epoch 6/1000
10/10 [==============================] - 0s 446us/step - loss: 1065418.7500
Epoch 7/1000
10/10 [==============================] - 0s 470us/step - loss: 701464.2500
Epoch 8/1000
10/10 [==============================] - 0s 459us/step - loss: 447053.3750
Epoch 9/1000
10/10 [==============================] - 0s 475us/step - loss: 275591.6562
Epoch 10/1000
10/10 [==============================] - 0s 438us/step - loss: 163943.1875
Epoch 11/1000
10/10 [==============================] - 0s 456us/step - loss: 93979.7344
Epoch 12/1000
10/10 [====

Epoch 1/1000
10/10 [==============================] - 0s 491us/step - loss: 328008.3438
Epoch 2/1000
10/10 [==============================] - 0s 433us/step - loss: 93312.5234
Epoch 3/1000
10/10 [==============================] - 0s 440us/step - loss: 9686.8047
Epoch 4/1000
10/10 [==============================] - 0s 424us/step - loss: 1581.7544
Epoch 5/1000
10/10 [==============================] - 0s 466us/step - loss: 3819.6165
Epoch 6/1000
10/10 [==============================] - 0s 432us/step - loss: 1442.4189
Epoch 7/1000
10/10 [==============================] - 0s 454us/step - loss: 86.3121
Epoch 8/1000
10/10 [==============================] - 0s 438us/step - loss: 128.2382
Epoch 9/1000
10/10 [==============================] - 0s 458us/step - loss: 89.0957
Epoch 10/1000
10/10 [==============================] - 0s 436us/step - loss: 19.9186
Epoch 11/1000
10/10 [==============================] - 0s 442us/step - loss: 20.8104
Epoch 12/1000
10/10 [==============================] - 0s 

Epoch 1/1000
10/10 [==============================] - 0s 565us/step - loss: 36840.5664
Epoch 2/1000
10/10 [==============================] - 0s 478us/step - loss: 2487.2317
Epoch 3/1000
10/10 [==============================] - 0s 657us/step - loss: 3802.8884
Epoch 4/1000
10/10 [==============================] - 0s 513us/step - loss: 310.4755
Epoch 5/1000
10/10 [==============================] - 0s 502us/step - loss: 489.0222
Epoch 6/1000
10/10 [==============================] - 0s 498us/step - loss: 60.3967
Epoch 7/1000
10/10 [==============================] - 0s 439us/step - loss: 51.3579
Epoch 8/1000
10/10 [==============================] - 0s 480us/step - loss: 15.9085
Epoch 9/1000
10/10 [==============================] - 0s 566us/step - loss: 3.2866
Epoch 10/1000
10/10 [==============================] - 0s 462us/step - loss: 2.8071
Epoch 11/1000
10/10 [==============================] - 0s 548us/step - loss: 0.5696
Epoch 12/1000
10/10 [==============================] - 0s 475us/step

In [10]:
print(V[0,:])

[ 1.41807612e+04  1.38785175e+04  1.31916689e+04  1.08478857e+04
  2.80444683e+03  8.43267665e+02  8.99683325e+01 -1.51266743e-01
  2.91378493e-01  0.00000000e+00]


In [11]:
print(S[0,:])

[[40.         40.85041278 39.63991294 40.41743992 40.2798528  38.43261923
  38.77978826 40.15126674 41.81934784 40.32685971]]


In [12]:
print(holdValues[0,:])

[ 1.41807612e+04  1.38785175e+04  1.31916689e+04  1.08478857e+04
  2.80444683e+03  8.43267665e+02  8.99683325e+01 -6.98525156e+00
  2.91378493e-01]
